In [1]:
import os
import io
import datetime
import logging
import sys

import numpy as np
import pandas as pd
from pandas.io.json import json_normalize 

import civis
import civis.io
from civis.futures import CivisFuture

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier, RandomForestRegressor
from civis.ml import ModelPipeline

import uuid
import json
from pprint import pprint
import tempfile
import concurrent.futures
from concurrent.futures import wait
from collections import namedtuple

/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
# Default feature lists for Rainbow Modeling Frame (each number corresponds to number of features)
feature_table = civis.io.read_civis_sql(sql='select * from bernie_nmarchio2.feature_list order by sort_order asc', use_pandas = True, database='Bernie 2020')
feature_list_large = list(feature_table[(feature_table['frame_large'] == 1)]['feature_name']) + ['state_code']
feature_list_medium = list(feature_table[(feature_table['frame_medium'] == 1)]['feature_name']) + ['state_code']
feature_list_small = list(feature_table[(feature_table['frame_small'] == 1)]['feature_name']) + ['state_code']

table_columns = civis.io.read_civis_sql(
    sql=f'''select ordinal_position as position, column_name, data_type 
    from information_schema.columns 
    where table_name = 'rainbow_modeling_frame' and table_schema = 'bernie_data_commons' and column_name != 'person_id'
    order by ordinal_position;''', use_pandas = True, database='Bernie 2020')

#exclusion_list_466 = [e for e in list(table_columns['column_name']) if e not in feature_list_466] 

In [9]:
# USER INPUT CELL

# DV table parameters
DATABASE = 'Bernie 2020'
# Primary key in both the DV table and the Modeling Frame
PRIMARY_KEY = 'person_id' 
# Table containing recoded Dependent Variables keyed to the PRIMARY_KEY
DV_TABLE = 'bernie_tryan.nh_gotc_dvs'
# List of binarized dependent variables (accepts 1, 0, and null values) in DV_TABLE
DV_LIST = ['like_bernie',
           'like_biden',
           'like_warren',
           'like_buttigieg',
           'like_other',
           'pro_bernie_1_id',
           'pro_bernie_1_2_id',
           'pro_bernie_fieldid_1_no_gop',
           'pro_bernie_fieldid_1']

# Modeling frame table parameters
# Table containing covariates and keyed to PRIMARY_KEY
MODELING_FRAME = 'bernie_data_commons.rainbow_modeling_frame'
# Columns in the Modeling Frame to exclude from feature list (i.e., strings or incomplete coverage)
EXCLUSION_COLUMNS = ['state_code',
                     'census_block_group_2010']

# Output table parameters
# Schema to contain prediction tables
SCHEMA = 'bernie_tryan'
# String that will be concatenated in front of the output table's name
PREFIX = 'gotp'

# Sampling parameters
# Non-response training data
    # True: automatically select people not in DV_TABLE at random from Phoenix (assumes person_id is PRIMARY_KEY)
    # False: automatically select people where the DV equals 0 from the DV_TABLE
SAMPLE_FROM_PHOENIX = False
# Number of non-response classes per target class (default is 2) 
CLASS_BALANCE = 2
# Maximum number of targets to randomly sample from DV_TABLE
MAX_TARGET_COUNT = 30000

In [10]:
datestamp = '{:%Y%m%d}'.format(datetime.date.today())

In [11]:
# Counts of positive classes
dv_sql_targets = "\n,".join(["sum({dv}) as {dv}".format(dv=i) for i in DV_LIST])
sql_collapse_targets = f"""select {dv_sql_targets} from {DV_TABLE};"""
sql_count_targets = civis.io.read_civis_sql(sql_collapse_targets, DATABASE)


In [12]:
# Determing training table proportion of positives to negatives (to avoid class imbalance problems)
sample_share = []
for i in range(len(DV_LIST)):
    if int(sql_count_targets[1][i]) > MAX_TARGET_COUNT:
        sql_count_targets[1][i] = MAX_TARGET_COUNT
    u = round(int(sql_count_targets[1][i])*CLASS_BALANCE)
    sample_share.append(u)      

In [13]:
for i in range(len(DV_LIST)):
    dv_item = DV_LIST[i]
    random_sample = sample_share[i]
    if SAMPLE_FROM_PHOENIX is True:
        zero_sample = f'''(select p.person_id, 0 as {dv_item} from phoenix_analytics.person p left join (select person_id from {DV_TABLE}) d on p.person_id = d.person_id where d.person_id is null and is_deceased = false and reg_record_merged = false and reg_on_current_file = true and reg_voter_flag = true order by random() limit {random_sample})'''   
    if SAMPLE_FROM_PHOENIX is False:
        zero_sample = f'''(select {PRIMARY_KEY}, {dv_item} from {DV_TABLE} where {dv_item} = 0 order by random() limit {random_sample})'''


In [14]:
# Create training tables
for i in range(len(DV_LIST)):
    if (int(sql_count_targets[1][i])*3) <= 1000:
        feature_select = "\n,".join(["{feature}".format(feature=f) for f in feature_list_small])
    if (int(sql_count_targets[1][i])*3) > 1000 & (int(sql_count_targets[1][i])*3) <= 2000:
        feature_select = "\n,".join(["{feature}".format(feature=f) for f in feature_list_medium])
    if (int(sql_count_targets[1][i])*3) > 2000:
        feature_select = "\n,".join(["{feature}".format(feature=f) for f in feature_list_large])
    dv_item = DV_LIST[i]
    print(dv_item)
    random_sample = sample_share[i]
    if SAMPLE_FROM_PHOENIX is True:
        zero_sample = f'''(select p.person_id, 0 as {dv_item} from phoenix_analytics.person p left join (select person_id from {DV_TABLE}) d on p.person_id = d.person_id where d.person_id is null and is_deceased = false and reg_record_merged = false and reg_on_current_file = true and reg_voter_flag = true order by random() limit {random_sample})'''   
    if SAMPLE_FROM_PHOENIX is False:
        zero_sample = f'''(select {PRIMARY_KEY}, {dv_item} from {DV_TABLE} where {dv_item} = 0 order by random() limit {random_sample})'''
    training_sql = f"""DROP TABLE IF EXISTS {SCHEMA}.{PREFIX}_training_{i} CASCADE;
    CREATE TABLE {SCHEMA}.{PREFIX}_training_{i} AS 
    (select * from (
    (select {PRIMARY_KEY}, {dv_item} from {DV_TABLE} where {dv_item} = 1 order by random() limit 30000) 
    union all 
    {zero_sample})
    inner join
    (select {PRIMARY_KEY}, {feature_select} from {MODELING_FRAME}) using({PRIMARY_KEY}));"""
    create_training_sql = civis.io.query_civis(training_sql, database=DATABASE)
    create_training_sql.result().state
    
    

like_bernie
like_biden
like_warren
like_buttigieg
like_other
pro_bernie_1_id
pro_bernie_1_2_id
pro_bernie_fieldid_1_no_gop
pro_bernie_fieldid_1


In [15]:
# Train models
train_list = []
model_list = []

for index, dv in enumerate(DV_LIST):
    print('TRAINING >>> {}'.format(dv))
    
    exc_list = DV_LIST.copy()
    exc_list.remove(dv)
    
    assert dv not in exc_list 
    
    name = f"""{dv}_{datestamp}"""
    model = ModelPipeline(model='sparse_logistic',
                          dependent_variable=dv,
                          primary_key=PRIMARY_KEY,
                          excluded_columns=EXCLUSION_COLUMNS,
                          #calibration='sigmoid',
                          model_name=name,
                          memory_requested=12000#,
                          #disk_requested=5
                         )
    
    where_string = '{} is not null'.format(dv)

    train = model.train(table_name=f"""{SCHEMA}.{PREFIX}_training_{index}""", 
                        database_name=DATABASE,
                        sql_where=where_string#,
                        #fit_params={'sample_weight': WEIGHT_VAR}
                       )
    
    model_list.append(model)
    train_list.append(train)    


TRAINING >>> like_bernie
TRAINING >>> like_biden
TRAINING >>> like_warren
TRAINING >>> like_buttigieg
TRAINING >>> like_other
TRAINING >>> pro_bernie_1_id
TRAINING >>> pro_bernie_1_2_id
TRAINING >>> pro_bernie_fieldid_1_no_gop
TRAINING >>> pro_bernie_fieldid_1


In [16]:
# Extract successful models
model_output = model_list
train_output = train_list

jobs_list = []
for t in train_output: 
    try:
        if len(t.metadata['output']) > 0:  
            jobs_list.append(t)
            print('Job success')
    except:
        print('Job failure')
        pass

    
model_output, train_output = zip(*((m, t) for m, t in zip(model_output, train_output) if t in jobs_list))
model_output = list(model_output)
train_output = list(train_output)

Job success
Job success
Job success
Job success
Job success
Job success
Job success
Job success
Job success


In [17]:
# Generate validation metrics
metrics_list = []

for a, b in enumerate(train_output):
    metric = {'job_id':b.job_id,
              'run_id':b.run_id,
              'dv': ''.join(b.metadata['run']['configuration']['data']['y']),
              'model': b.metadata['model']['model'],
              'time_of_train_run': b.metadata['run']['time_of_run'],
              'n_rows': b.metadata['data']['n_rows'],
              'n_features': b.metadata['data']['n_cols'],
              'auc': b.metadata['metrics']['roc_auc'],
              'deciles': b.metadata['metrics']['deciles'],
              'confusion_matrix': b.metadata['metrics']['confusion_matrix'],
              'accuracy': b.metadata['metrics']['accuracy'],
              'p_correct': b.metadata['metrics']['p_correct'],
              'pop_incidence_true': b.metadata['metrics']['pop_incidence_true'],
              'feature_list':b.metadata['model']['parameters']['relvars'],
              'feature_coefficients':b.metadata['model']['parameters']['coef'],
              'feature_importance':b.metadata['model']['parameters']
             }
    metrics_list.append(metric)
    
metric_order = (['job_id', 'run_id', 'dv', 'model', 'time_of_train_run', 'n_rows', 'n_features',
                 'auc', 'deciles', 'confusion_matrix', 'accuracy', 'p_correct','pop_incidence_true',
                 'feature_list','feature_coefficients','feature_importance'])

validation_df = pd.DataFrame.from_records(metrics_list, columns=metric_order, index='run_id')
validation_df

,job_id,dv,model,time_of_train_run,n_rows,n_features,auc,deciles,confusion_matrix,accuracy,p_correct,pop_incidence_true,feature_list,feature_coefficients,feature_importance
run_id,,,,,,,,,,,,,,,
201343221,57433455,like_bernie,sparse_logistic,2020-01-16T22:33:44Z,1953,501,0.693335,"[0.1076923076923077, 0.14795918367346939, 0.21...","[[1133, 162], [439, 219]]",0.692268,"[0.8749034749034749, 0.33282674772036475]","[0.6630824372759857, 0.33691756272401435]","[civis_2018_ballot_dropoff, civis_2018_congres...","[[0.9149689342891341, 15.892652511865366, 5.08...","{'relvars': ['civis_2018_ballot_dropoff', 'civ..."
201343242,57433465,like_biden,sparse_logistic,2020-01-16T22:34:24Z,3640,501,0.694258,"[0.12087912087912088, 0.17307692307692307, 0.2...","[[2122, 280], [877, 361]]",0.682143,"[0.8834304746044963, 0.29159935379644586]","[0.6598901098901099, 0.3401098901098901]","[civis_2020_partisanship, dnc_2018_income_rank...","[[0.411775600306873, -0.0018824508372330413, 2...","{'relvars': ['civis_2020_partisanship', 'dnc_2..."
201343364,57433478,like_warren,sparse_logistic,2020-01-16T22:33:45Z,2263,501,0.666839,"[0.0752212389380531, 0.1277533039647577, 0.274...","[[1354, 145], [635, 129]]",0.655325,"[0.9032688458972649, 0.168848167539267]","[0.6623950508174989, 0.3376049491825011]","[civis_2020_ideology_liberal, civis_2018_medic...","[[1.2641077946076116, -12.801411232318806, 1.1...","{'relvars': ['civis_2020_ideology_liberal', 'c..."
201343380,57433531,like_buttigieg,sparse_logistic,2020-01-16T22:35:03Z,1975,501,0.624763,"[0.08585858585858586, 0.25380710659898476, 0.3...","[[1142, 150], [560, 123]]",0.640506,"[0.8839009287925697, 0.1800878477306003]","[0.6541772151898734, 0.34582278481012657]","[civis_2020_ideology_liberal, dnc_2018_college...","[[-0.25556355221575644, 0.7559829109687528, 0....","{'relvars': ['civis_2020_ideology_liberal', 'd..."
201343401,57433541,like_other,sparse_logistic,2020-01-16T22:35:10Z,2977,501,0.563232,"[0.21476510067114093, 0.3265993265993266, 0.32...","[[1767, 172], [903, 135]]",0.638898,"[0.9112944816915935, 0.13005780346820808]","[0.6513268390997649, 0.34867316090023515]","[civis_2018_gotv, civis_2020_subeth_puerto_ric...","[[-0.42931665434960253, -3.4950113096817277, 1...","{'relvars': ['civis_2018_gotv', 'civis_2020_su..."
201343445,57433579,pro_bernie_1_id,sparse_logistic,2020-01-16T22:34:55Z,1318,297,0.702650,"[0.13636363636363635, 0.1893939393939394, 0.18...","[[757, 115], [272, 174]]",0.706373,"[0.8681192660550459, 0.3901345291479821]","[0.661608497723824, 0.33839150227617604]","[civis_2018_ballot_dropoff, civis_2018_congres...","[[0.9807083438641834, 1.8268499495522532, 1.38...","{'relvars': ['civis_2018_ballot_dropoff', 'civ..."
201343454,57433589,pro_bernie_1_2_id,sparse_logistic,2020-01-16T22:35:12Z,1983,501,0.703901,"[0.07575757575757576, 0.20603015075376885, 0.2...","[[1174, 150], [418, 241]]",0.713565,"[0.8867069486404834, 0.36570561456752654]","[0.6676752395360565, 0.33232476046394355]","[civis_2018_ballot_dropoff, civis_2018_congres...","[[1.1247037208263757, 10.612770679382997, 1.14...","{'relvars': ['civis_2018_ballot_dropoff', 'civ..."
201343467,57433601,pro_bernie_fieldid_1_no_gop,sparse_logistic,2020-01-16T22:39:28Z,66480,501,0.675982,"[0.1072503008423586, 0.18170878459687123, 0.23...","[[40888, 3432], [17569, 4591]]",0.684100,"[0.9225631768953069, 0.20717509025270758]","[0.6666666666666666, 0.3333333333333333]","[civis_2020_marriage, civis_2020_partisanship,...","[[-0.026157994320820326, 0.13184472730305638, ...","{'relvars': ['civis_2020_marriage', 'civis_202..."
201343478,57433609,pro_bernie_fieldid_1,sparse_logistic,2020-01-16T22:39:18Z,66480,501,0.676041,"[0.10273766546329723, 0.17854993983152828, 0.2...","[[40780, 3540], [17491, 4669]]",0.683649,"[0.9201263537906137, 0.21069494584837545]","[0.6666666666666666, 0.3333333333333333]","[civis_2020_marriage, civis_2020_partisanship,...","[[-0.031152872007577045, 0.23180272152514003, ...","{'relvars': ['civis_2020_marriage', 'civis_202.

In [18]:
# Write validation metrics to Redshift
create_validation_table = civis.io.dataframe_to_civis(df=validation_df,
                                                 database=DATABASE, 
                                                 table= f'{SCHEMA}.{PREFIX}_validation_{datestamp}', 
                                                 existing_table_rows='drop')


In [19]:
# Score the voterfile
scores_list = []
for m,t in zip(model_output, train_output):
    DV_NAME = ''.join(t.metadata['run']['configuration']['data']['y'])
    print(DV_NAME)
    SCORES_TABLE = f'{SCHEMA}.{PREFIX}_{DV_NAME}_{datestamp}'
    scores_list.append(SCORES_TABLE)
    scores = m.predict(primary_key=PRIMARY_KEY,
                       database_name=DATABASE, 
                       table_name=MODELING_FRAME,
                       sql_where="state_code = 'NH'",
                       if_exists='drop',
                       output_table=SCORES_TABLE,
                       disk_space=10)
scores.result()


like_bernie
like_biden
like_warren
like_buttigieg
like_other
pro_bernie_1_id
pro_bernie_1_2_id
pro_bernie_fieldid_1_no_gop
pro_bernie_fieldid_1


{'container_id': 57436574,
 'error': None,
 'finished_at': '2020-01-16T23:31:21.000Z',
 'id': 201347933,
 'is_cancel_requested': False,
 'started_at': '2020-01-16T22:47:05.000Z',
 'state': 'succeeded'}

In [20]:
# Generate SQL for final output table and drop intermediary tables
input_train_list = []
output_score_list = []
for i in range(len(DV_LIST)):
    input_train = f"{SCHEMA}.{PREFIX}_training_{i}"
    input_train_list.append(input_train)
    output_score = f"{SCHEMA}.{PREFIX}_{DV_LIST[i]}_{datestamp}"
    output_score_list.append(output_score)

drop_input_train_sql = "\n".join(["drop table if exists {tbl};".format(tbl=v) for v in input_train_list])
drop_output_score_sql = "\n".join(["drop table if exists {tbl};".format(tbl=t) for t in output_score_list])  
dv_strings = "\n,".join(["{dv_score}_1 as {dv_score}".format(dv_score=dv) for dv in DV_LIST])
dv_tiles = "\n,".join(["NTILE(100) OVER (ORDER BY {dv_tile}_1) AS {dv_tile}_100".format(dv_tile=dv) for dv in DV_LIST])
join_table = []
if len(output_score_list) > 1:
    for i in output_score_list[1:]:
        j = str(' left join '+f'{i}'+f' using({PRIMARY_KEY}) ')
        join_table.append(j)
        #dv_strings = "\nleft join ".join(["{dv_score}".format(table=tbl) for tbl in table_list[i])


In [21]:
output_table_sql = f"""
set query_group to 'importers';
set wlm_query_slot_count to 3;
DROP TABLE IF EXISTS {SCHEMA}.{PREFIX}_output_{datestamp};
CREATE TABLE {SCHEMA}.{PREFIX}_output_{datestamp}
  DISTSTYLE KEY
  DISTKEY ({PRIMARY_KEY})
  SORTKEY ({PRIMARY_KEY})
  AS ("""+'select '+ f"{PRIMARY_KEY} \n," + dv_strings + "\n," + dv_tiles + ' from '+ ''.join(output_score_list[0]) + ''.join(join_table) +');'  


In [22]:
print(output_table_sql)


set query_group to 'importers';
set wlm_query_slot_count to 3;
DROP TABLE IF EXISTS bernie_tryan.gotp_output_20200116;
CREATE TABLE bernie_tryan.gotp_output_20200116
  DISTSTYLE KEY
  DISTKEY (person_id)
  SORTKEY (person_id)
  AS (select person_id 
,like_bernie_1 as like_bernie
,like_biden_1 as like_biden
,like_warren_1 as like_warren
,like_buttigieg_1 as like_buttigieg
,like_other_1 as like_other
,pro_bernie_1_id_1 as pro_bernie_1_id
,pro_bernie_1_2_id_1 as pro_bernie_1_2_id
,pro_bernie_fieldid_1_no_gop_1 as pro_bernie_fieldid_1_no_gop
,pro_bernie_fieldid_1_1 as pro_bernie_fieldid_1
,NTILE(100) OVER (ORDER BY like_bernie_1) AS like_bernie_100
,NTILE(100) OVER (ORDER BY like_biden_1) AS like_biden_100
,NTILE(100) OVER (ORDER BY like_warren_1) AS like_warren_100
,NTILE(100) OVER (ORDER BY like_buttigieg_1) AS like_buttigieg_100
,NTILE(100) OVER (ORDER BY like_other_1) AS like_other_100
,NTILE(100) OVER (ORDER BY pro_bernie_1_id_1) AS pro_bernie_1_id_100
,NTILE(100) OVER (ORDER BY pro_

In [23]:
# Create final output table
create_output_table = civis.io.query_civis(sql=output_table_sql, database=DATABASE)
create_output_table.result().state


'succeeded'

In [24]:
# Drop intermediary tables
drop_input_train_query = civis.io.query_civis(sql=drop_input_train_sql, database=DATABASE)
drop_input_train_query.result().state

drop_output_score_query = civis.io.query_civis(sql=drop_output_score_sql, database=DATABASE)
drop_output_score_query.result().state

'succeeded'

In [25]:
print(drop_input_train_sql)
print(drop_output_score_sql)

drop table if exists bernie_tryan.gotp_training_0;
drop table if exists bernie_tryan.gotp_training_1;
drop table if exists bernie_tryan.gotp_training_2;
drop table if exists bernie_tryan.gotp_training_3;
drop table if exists bernie_tryan.gotp_training_4;
drop table if exists bernie_tryan.gotp_training_5;
drop table if exists bernie_tryan.gotp_training_6;
drop table if exists bernie_tryan.gotp_training_7;
drop table if exists bernie_tryan.gotp_training_8;
drop table if exists bernie_tryan.gotp_like_bernie_20200116;
drop table if exists bernie_tryan.gotp_like_biden_20200116;
drop table if exists bernie_tryan.gotp_like_warren_20200116;
drop table if exists bernie_tryan.gotp_like_buttigieg_20200116;
drop table if exists bernie_tryan.gotp_like_other_20200116;
drop table if exists bernie_tryan.gotp_pro_bernie_1_id_20200116;
drop table if exists bernie_tryan.gotp_pro_bernie_1_2_id_20200116;
drop table if exists bernie_tryan.gotp_pro_bernie_fieldid_1_no_gop_20200116;
drop table if exists berni

In [26]:
# Grant team on tables
grant_statement = f"""
GRANT ALL ON SCHEMA {SCHEMA} TO GROUP bernie_data;
GRANT SELECT ON {SCHEMA}.{PREFIX}_output_{datestamp} TO GROUP bernie_data;
GRANT SELECT ON {SCHEMA}.{PREFIX}_validation_{datestamp} TO GROUP bernie_data;
"""
grant_team = civis.io.query_civis(sql=grant_statement, database=DATABASE)
grant_team.result().state

'succeeded'

In [27]:
print(grant_statement)


GRANT ALL ON SCHEMA bernie_tryan TO GROUP bernie_data;
GRANT SELECT ON bernie_tryan.gotp_output_20200116 TO GROUP bernie_data;
GRANT SELECT ON bernie_tryan.gotp_validation_20200116 TO GROUP bernie_data;

